### Local RAG Workflow with Db2 and llama.cpp

This notebook demonstrates a complete Retrieval-Augmented Generation (RAG) pipeline running on a local system:

* **Embeddings** are generated locally using the Granite model served via `llama.cpp`
* **Vector search** is performed in Db2 using built-in `VECTOR` functions
* **Context retrieval** and **prompt construction** are handled locally
* Only the **final text generation** step uses a hosted LLM (Mistral) via Watsonx.ai

By keeping embedding generation and prompting local, the workflow reduces latency, avoids cloud dependency for sensitive data, and offers more control over the overall process.

**Setup and Requirements**
For installation steps and additional context, see the accompanying [README.md](./README.md).


## Step 0: Imports

In [1]:
# Core libraries
import os
import pandas as pd

# Environment variables
from dotenv import load_dotenv, dotenv_values

# NLP and embeddings
import spacy
import trafilatura
from llama_cpp import Llama

# Watsonx AI
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams, EmbedTextParamsMetaNames

# LangChain components
from langchain_ibm import WatsonxLLM
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader
# from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate

# Notebook display utilities
from IPython.display import display, Markdown

if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')

%run db2.ipynb

<>:1708: SyntaxWarning: invalid escape sequence '\s'
<>:2305: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_389282/2299624180.py:1708: SyntaxWarning: invalid escape sequence '\s'
  firstCommand = "(?:^\s*)([a-zA-Z]+)(?:\s+.*|$)"
/tmp/ipykernel_389282/2299624180.py:2305: SyntaxWarning: invalid escape sequence '\?'
  pattern = "\?\*[0-9]+"


Db2 Extensions Loaded. Version: 2024-09-16


## Step 1: Web Content Extraction

In [ ]:
url = 'https://community.ibm.com/community/user/blogs/shaikh-quader/2024/05/07/building-an-in-db-linear-regression-model-with-ibm'
downloaded = trafilatura.fetch_url(url)

if downloaded:
    article = trafilatura.extract(downloaded)
    print(article[:1000])  # Preview first 1000 chars
else:
    print("Failed to fetch content.")

Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin.
The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this link.

## Step 2: Chunking the Clean Text for Embedding

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
def overlapping_sentence_chunker(text, max_words=200, overlap_words=50):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    
    chunks = []
    current_chunk = []
    current_length = 0

    i = 0
    while i < len(sentences):
        sentence = sentences[i]
        sentence_length = len(sentence.split())

        if current_length + sentence_length <= max_words:
            current_chunk.append(sentence)
            current_length += sentence_length
            i += 1
        else:
            chunks.append(" ".join(current_chunk))
            # Start new chunk with overlap
            overlap = []
            overlap_len = 0
            j = len(current_chunk) - 1
            while j >= 0 and overlap_len < overlap_words:
                s = current_chunk[j]
                overlap.insert(0, s)
                overlap_len += len(s.split())
                j -= 1
            current_chunk = overlap
            current_length = overlap_len

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [5]:
chunks = overlapping_sentence_chunker(article, max_words=200, overlap_words=50)
print(f"{len(chunks)} chunks created.")
print(chunks[0])

15 chunks created.
Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is availa

## Generating Embedding Vectors

In [6]:
embedding_model = Llama(model_path="models/granite-embedding-30m-english-Q6_K.gguf", embedding=True, verbose=False)

In [7]:
embeddings = embedding_model.create_embedding(chunks)

In [8]:
for text, item in zip(chunks, embeddings["data"]):
    print(f"Text: {text}")
    print("Embedding (partial):", item["embedding"][:12], "...\n")

Text: Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this

## Inserting Vectors into Db2

In [9]:
db2creds = dotenv_values('.env')
%sql CONNECT CREDENTIALS db2creds

%sql DROP TABLE IF EXISTS embeddings

SQL_CREATE_TABLE = """
CREATE TABLE embeddings (
    id INT NOT NULL GENERATED ALWAYS AS IDENTITY 
        (START WITH 1, INCREMENT BY 1),
    content CLOB,
    embedding VECTOR(384, FLOAT32),
    PRIMARY KEY (id)
)
"""
%sql {SQL_CREATE_TABLE}

# Prepare values: list of tuples (content, embedding_vector_str)
values = []
for i, (text, item) in enumerate(zip(chunks, embeddings["data"])):
    embedding = item.get("embedding")
    if embedding and len(embedding) == 384:
        vector_str = "[" + ", ".join(map(str, embedding)) + "]"
        values.append((text, vector_str))
    else:
        print(f"Skipping row {i+1}: invalid embedding")

# Prepare SQL statement with VECTOR function
stmt = %sql prepare INSERT INTO embeddings (content, embedding) VALUES (?, VECTOR(?, 384, FLOAT32))

# Disable autocommit
%sql autocommit off

# Execute prepared insert statement using magic-style loop
for content, vector_str in values:
    %sql execute :stmt using :content, :vector_str

# Commit the work
%sql commit work

# Enable autocommit back
%sql autocommit on

Connection successful. SAMPLE @ localhost 
Command completed.
Command completed.


# Ask LLM

In [10]:
question = "How to build a linear regression model using IDAX?"
# question = 'How to see the list of in database ML models in Db2?'
# question = 'How to impute missing values of columns in Db2?'
# query = 'What is Python UDF?'

embedding = embedding_model.create_embedding(question)
query_vector = embedding['data'][0]['embedding']

## Vector Search and Context Retrieval (Db2)

In [11]:
# Step 1: Format query embedding for Db2
query_embedding_str = '[' + ', '.join(map(str, query_vector)) + ']'
top_k = 5  # You can adjust this number

# Step 2: SQL to get top-k most similar chunks based on Euclidean distance
SQL_DISTANCE = f"""
SELECT 
    content AS CONTEXT,
    VECTOR_DISTANCE(
        VECTOR('{query_embedding_str}', 384, FLOAT32),
        embedding,
        EUCLIDEAN
    ) AS DISTANCE
FROM embeddings
ORDER BY DISTANCE ASC
FETCH FIRST {top_k} ROWS ONLY
"""

# Step 3: Run the query with db2magic and store the result in a DataFrame
result_df = %sql {SQL_DISTANCE}

# Step 4: Combine all top-k text chunks into a single context string
context_str = "\n\n".join(result_df["CONTEXT"].tolist())

## LLM setup

In [12]:


load_dotenv(os.getcwd()+"/.env", override=True)

# llm for generating responses
def get_llm():
    # model_id = 'meta-llama/llama-3-1-70b-instruct'
    model_id = 'mistralai/mistral-large'
    parameters = {
        GenParams.MAX_NEW_TOKENS: 512,
        GenParams.TEMPERATURE: 0.6,
    }
   
    watsonx_llm = WatsonxLLM(
        model_id=model_id,
        url="https://us-south.ml.cloud.ibm.com",
        apikey=os.getenv("WATSONX_APIKEY", ""),
        project_id=os.getenv("WATSONX_PROJECT", ""),
        params=parameters,
    )
    return watsonx_llm
    
llm = get_llm()

## Prompt Template

In [13]:
prompt_template = """
You are a knowledgeable assistant. Answer the question based solely on the provided context.
- If the context contains the answer, respond directly to the reader using 'you' to make it personal.
- If the answer includes code, provide an explanation of the code following the code block.
- If the information is not available in the context, respond with 'The information is not available in the provided context.'

Context:
{context}

Question:
{question}

Answer:
""".strip()

### Generate an Answer Using wx.ai

In [14]:
# Custom prompt template (no LangChain needed)
prompt_template = """
You are a knowledgeable assistant. Answer the question based solely on the provided context.
- If the context contains the answer, respond directly to the reader using 'you' to make it personal.
- If the answer includes code, provide an explanation of the code following the code block.
- If the information is not available in the context, respond with 'The information is not available in the provided context.'

Context:
{context}

Question:
{question}

Answer:
""".strip()

# Fill the template
prompt = prompt_template.format(context=context_str, question=question)

# Call Watsonx LLM
response = llm(prompt)

# Display the result
from IPython.display import Markdown, display

display(Markdown(f"**Q:** {question}\n\n**A:** {response}"))

**Q:** How to build a linear regression model using IDAX?

**A:** 
To build a linear regression model using IDAX, you can use the `LINEAR_REGRESSION` stored procedure (SP) provided by IBM Db2. Here is the SQL call to train a linear regression model:

```sql
CALL IDAX.LINEAR_REGRESSION('model=GOSALES.GOSALES_LINREG, intable=GOSALES.GOSALES_TRAIN, id=ID, target=PURCHASE_AMOUNT, incolumn=AGE;GENDER;MARITAL_STATUS;PROFESSION, intercept=true');
```

This code specifies the parameters for the linear regression model, including the table containing the training data (`intable=GOSALES.GOSALES_TRAIN`), the target column (`target=PURCHASE_AMOUNT`), and the input features (`incolumn=AGE;GENDER;MARITAL_STATUS;PROFESSION`). The `intercept=true` parameter means that the model will learn the value of the intercept.

After the training completes, the new model `GOSALES_LINREG` will be added to Db2’s model catalog. You can list the existing models in the catalog with the following call:

```sql
CALL IDAX.LIST_MODELS('format=short, all=true');
```

Additionally, the learned values of the intercept and the coefficients, along with other learned parameter values, are saved in a table named `GOSALES_LINREG_MODEL`. You can display these values using the following SQL query:

```sql
SELECT VAR_NAME, LEVEL_NAME, VALUE FROM GOSALES.GOSALES_LINREG_MODEL;
```

This process allows you to build and manage a linear regression model directly within the Db2 database.